## Description
> Data set has over 183 milion rows and it is impossible to process in a regular laptop at one go. Therefore processing completed in chunks

#### dependencies

In [3]:
import pandas as pd
import numpy as np
import random

#### process skewness

In [4]:
def norm_(column):
    column =np.log(column)
    return column

#### extract features

In [41]:
def process_data(data_sample):
    columns = data_sample.columns
    if 'click_id' in columns:
        test_set=1
    else:
        test_set=0
    data_sample = data_sample.sort_values(by=['click_time'])  
    data_sample['day']=pd.to_datetime(data_sample.click_time).dt.day.astype('int')
    data_sample['hour']=pd.to_datetime(data_sample.click_time).dt.hour.astype('int')
    data_sample['min']=pd.to_datetime(data_sample.click_time).dt.minute.astype('int')
   
    #app_counts
    app_count = data_sample[['ip','app','hour','day']].groupby(['ip','hour','day']).app.nunique()
    app_count = app_count.reset_index()
    app_count.columns = ['ip','hour','day','app_count']
    app_count['app_count'] = norm_(app_count['app_count'])
    merged = pd.merge(data_sample, app_count, how='left', on = ['ip','hour','day'])
    del app_count
    del data_sample
    
    # number of given app seen in tht hour
    app_count = merged[['ip','app','hour','day']].groupby(['app','hour','day']).count()
    app_count = app_count.reset_index()
    app_count.columns = ['app','hour','day','apps']
    app_count['apps'] = norm_(app_count['apps'])
    merged = pd.merge(merged, app_count, how='left', on = ['app','hour','day'])
    del app_count
   
    # app min
    
    app_count = merged[['ip','app','hour','day','min']].groupby(['ip','hour','day','min']).app.nunique()
    app_count = app_count.reset_index()
    app_count.columns = ['ip','hour','day','min','app_count_min']
    app_count['app_count_min'] = norm_(app_count['app_count_min'])
    merged = pd.merge(merged, app_count, how='left', on = ['ip','hour','day','min'])
    del app_count
    
    
    #device counts
    device_count = merged[['ip','device','hour','day']].groupby(['ip','hour','day']).device.nunique()
    device_count = device_count.reset_index()
    device_count.columns = ['ip','hour','day','device_count']
    device_count['device_count'] = norm_(device_count['device_count'])
    merged = pd.merge(merged, device_count, how='left', on = ['ip','hour','day'])
    del device_count
    
    # num devices seen that hour
    device_count = merged[['ip','device','hour','day']].groupby(['device','hour','day']).count()
    device_count = device_count.reset_index()
    device_count.columns = ['device','hour','day','devices']
    device_count['devices'] = norm_(device_count['devices'])
    merged = pd.merge(merged, device_count, how='left', on = ['device','hour','day'])
    del device_count
    
    #device min
    device_count = merged[['ip','device','hour','day','min']].groupby(['ip','hour','day','min']).device.nunique()
    device_count = device_count.reset_index()
    device_count.columns = ['ip','hour','day','min','device_count_min']
    device_count['device_count_min'] = norm_(device_count['device_count_min'])
    merged = pd.merge(merged, device_count, how='left', on = ['ip','hour','day','min'])
    del device_count
    
    #os_counts
    os_count = merged[['ip','os','hour','day']].groupby(['ip','hour','day']).os.nunique()
    os_count = os_count.reset_index()
    os_count.columns = ['ip','hour','day','os_count']
    os_count['os_count'] = norm_(os_count['os_count'])
    merged = pd.merge(merged, os_count, how='left', on = ['ip','hour','day'])
    del os_count
    
    # os seen that hour
    os_count = merged[['ip','os','hour','day']].groupby(['os','hour','day']).count()
    os_count = os_count.reset_index()
    os_count.columns = ['os','hour','day','oss']
    os_count['oss'] = norm_(os_count['oss'])
    merged = pd.merge(merged, os_count, how='left', on = ['os','hour','day'])
    del os_count
    
    #os_counts min
    os_count = merged[['ip','os','hour','day','min']].groupby(['ip','hour','day','min']).os.nunique()
    os_count = os_count.reset_index()
    os_count.columns = ['ip','hour','day','min','os_count_min']
    os_count['os_count_min'] = norm_(os_count['os_count_min'])
    merged = pd.merge(merged, os_count, how='left', on = ['ip','hour','day','min'])
    del os_count

    #channel count
    channel_count = merged[['ip','channel','hour','day']].groupby(['ip','hour','day']).channel.nunique()
    channel_count = channel_count.reset_index()
    channel_count.columns = ['ip','hour','day','channel_count']
    channel_count['channel_count'] = norm_(channel_count['channel_count'])
    merged = pd.merge(merged, channel_count, how='left', on = ['ip','hour','day'])
    del channel_count
    
    # channel seen tht hour
    channel_count = merged[['ip','channel','hour','day']].groupby(['channel','hour','day']).count()
    channel_count = channel_count.reset_index()
    channel_count.columns = ['channel','hour','day','channels']
    channel_count['channels'] = norm_(channel_count['channels'])
    merged = pd.merge(merged, channel_count, how='left', on = ['channel','hour','day'])
    del channel_count
    
    #channel min
    channel_count = merged[['ip','channel','hour','day','min']].groupby(['ip','hour','day','min']).channel.nunique()
    channel_count = channel_count.reset_index()
    channel_count.columns = ['ip','hour','day','min','channel_count_min']
    channel_count['channel_count_min'] = norm_(channel_count['channel_count_min'])
    merged = pd.merge(merged, channel_count, how='left', on = ['ip','hour','day','min'])
    del channel_count
    
    # app device os channel combo count
    adoc = merged[['ip','app','device','os','channel']].groupby(['app','device','os','channel']).count()
    adoc = adoc.reset_index()
    adoc.columns = ['app','device','os','channel','adoc']
    adoc['adoc'] = norm_(adoc['adoc'])
    merged = pd.merge(merged,adoc,how='left',on=['app','device','os','channel'])
    del adoc
    
    # app device os channel combo count min
    adoc = merged[['ip','app','device','os','channel','day','hour','min']].groupby(['app','device','os',
                                                                                    'channel','day','hour','min']).count()
    adoc = adoc.reset_index()
    adoc.columns = ['app','device','os','channel','day','hour','min','adoc_min']
    adoc['adoc_min'] = norm_(adoc['adoc_min'])
    merged = pd.merge(merged,adoc,how='left',on=['app','device','os','channel','day','hour','min'])
    del adoc
    
    # click counts
    click_count = merged[['ip','channel','hour','day']].groupby(['ip','hour','day']).count()
    click_count = click_count.reset_index()
    click_count.columns = ['ip','hour','day','click_count']
    click_count['click_count']=norm_(click_count['click_count'])
    merged = pd.merge(merged, click_count, how='left', on = ['ip','hour','day'])
    del click_count
    
    # click counts MIN
    click_count = merged[['ip','channel','hour','day','min']].groupby(['ip','hour','day','min']).count()
    click_count = click_count.reset_index()
    click_count.columns = ['ip','hour','day','min','click_count_min']
    click_count['click_count_min'] = norm_(click_count['click_count_min'])
    merged = pd.merge(merged, click_count, how='left', on = ['ip','hour','day','min'])
    del click_count
    
    #click counts cumcount min
    merged['click_count_cumsum_min'] = merged[['ip','channel','hour','day','min']].groupby(['ip','day','hour','min']).channel.cumcount()
    

    #click counts cumsum hour
    merged['click_count_cumsum'] = merged[['ip','channel','hour','day']].groupby(['ip','day','hour']).channel.cumcount()
    
    if test_set==0:
        features = merged[['ip','app','device','os','channel','hour','min','day','app_count','device_count','os_count',
                           'channel_count','click_count','apps','devices','oss',
                           'channels','adoc','app_count_min','device_count_min','os_count_min','channel_count_min',
                           'click_count_min','adoc_min','click_count_cumsum','click_count_cumsum_min',
                           'is_attributed']]
    else:
        features = merged[['click_id','ip','app','device','os','channel','hour','min','day','app_count','device_count','os_count',
                           'channel_count','click_count','apps','devices','oss',
                           'channels','adoc','app_count_min','device_count_min','os_count_min','channel_count_min',
                           'click_count_min','adoc_min','click_count_cumsum','click_count_cumsum_min',
                           'is_attributed']]
    return features

#### implement feature extraction on training data

In [ ]:
chunksize = 10 ** 6
k=0
for chunk in pd.read_csv('train.csv', chunksize=chunksize):
    chunk = process_data(chunk)
    df1 = chunk[chunk.is_attributed==1]
   # df3 = df1.copy()
    n = len(df1)
    df2 = chunk[chunk.is_attributed==0]
    df2 = df2.sample(n*6)
    frames = [df1,df2]
    con_f = pd.concat(frames)
    con_f =con_f.sample(frac=1.0)
    data_sample = con_f
    del con_f
    del df1
    del df2
    del frames
    
    k+=1
    print(k)
    print(len(data_sample))
    print(len(data_sample[data_sample.is_attributed==1]))
    
    
    
    data_sample.to_pickle('.\processed_train_parts1\part'+str(k)+'.pkl')

1
11851
1693
2
11865
1695
3
12019
1717
4
12859
1837
5
14084
2012
6
16177
2311
7
14070
2010
8
13055
1865
9
11837
1691
10
13202
1886
11
15932
2276
12
16779
2397
13
18032
2576
14
21028
3004
15
21630
3090
16
22876
3268
17
25151
3593
18
24815
3545
19
25151
3593
20
23835
3405
21
23765
3395
22
19824
2832
23
16681
2383
24
16079
2297
25
15260
2180
26
16352
2336
27
15631
2233
28
17521
2503
29
17255
2465
30
18445
2635
31
18053
2579
32
18074
2582
33
19271
2753
34
19075
2725
35
18592
2656
36
18410
2630
37
22869
3267
38
22540
3220
39
21714
3102
40
20979
2997
41
19516
2788
42
17297
2471
43
17409
2487
44
16926
2418
45
18487
2641
46
20545
2935
47
21147
3021
48
21840
3120
49
21266
3038


#### extract features test set

In [5]:
#data_sample = pd.read_csv('test.csv')
#data_sample['is_attributed']=2
#print(len(data_sample))
#features = process_data(data_sample)
#features['click_id']=data_sample['click_id']    
    
#features.to_csv('.\processed_test_parts\processed_test_data_2.csv')

chunksize = 10 ** 6
k=0
for chunk in pd.read_csv('test.csv', chunksize=chunksize):
    chunk['is_attributed'] = 2
    features = process_data(chunk)
    k+=1
    print(k)
    print(len(data_sample))
    features.to_pickle('.\processed_test_parts\part'+str(k)+'.pkl')

18790469
